# Muskingum routing in the IJssel - verification script

Code adjusted from https://github.com/quaquel/epa1361_open.
The output of this model is equal to my own model, as can be seen in [this script](ijssel_muskingum.ipynb).

Source: https://onlinelibrary.wiley.com/doi/abs/10.1111/jfr3.12532

Ciullo, A., de Bruijn, K. M., Kwakkel, J. H., & Klijn, F. (2019). Accounting for the uncertain effects of hydraulic interactions in optimising embankments heights: Proof of principle for the IJssel River. Journal of Flood Risk Management, e12532.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import generate_network
from functions_ijssel_muskingum import Muskingum

In [ ]:
G, dike_list = generate_network.get_network()

In [ ]:
class DikeNetwork(object):
    def __init__(self):
        # planning steps
        self.num_events = 30
        
        # load network
        G, dike_list = generate_network.get_network()

        self.Qpeaks = 2000 #np.random.uniform(1000,16000,100)
        
        self.G = G
        self.dikelist = dike_list
        
    def printG(self):
        print(G.nodes.data())
        
    def getG(self):
        return G
        
        
    def init_node(self,value, time):
        init = np.repeat(value, len(time)).tolist()
        return init

    def _initialize_hydroloads(self, node, time, Q_0):
        #node['cumVol'], node['wl'], node['Qpol'], node['hbas'] = (
        #    self.init_node(0, time) for _ in range(4))
        node['Qin'], node['Qout'] = (self.init_node(Q_0, time) for _ in range(2))
        #node['status'] = self.init_node(False, time)
        #node['tbreach'] = np.nan
        return node
    
    def calc_wave(self,timestep=1):
        startnode = G.node['A.0']
        waveshape_id = 0
        Qpeak = self.Qpeaks#[0]
        dikelist = self.dikelist
        time = np.arange(0, startnode['Qevents_shape'].loc[waveshape_id].shape[0],
                             timestep)
        startnode['Qout'] = Qpeak * startnode['Qevents_shape'].loc[waveshape_id]
        
        # Initialize hydrological event:
        for key in dikelist:
            node = G.node[key] 
            #Q_0 = int(G.node['A.0']['Qout'][0])
            Q_0 = G.node['A.0']['Qout'][0]
            self._initialize_hydroloads(node, time, Q_0)
            
        # Run the simulation:
        # Run over the discharge wave:
        for t in range(1, len(time)):
            # Run over each node of the branch:
            for n in range(0, len(dikelist)):
                # Select current node:
                node = G.node[dikelist[n]]
                if node['type'] == 'dike':
                    # Muskingum parameters:
                    C1 = node['C1']
                    C2 = node['C2']
                    C3 = node['C3']
                    
                    prev_node = G.node[node['pnode']]
                    # Evaluate Q coming in a given node at time t:
                    node['Qin'][t] = Muskingum(C1, C2, C3,
                                                   prev_node['Qout'][t],
                                                   prev_node['Qout'][t - 1],
                                                   node['Qin'][t - 1])
                     
                    node['Qout'][t] = node['Qin'][t]
                    
    def __call__(self, timestep=1, **kwargs):
        G = self.G
        Qpeaks = self.Qpeaks
        dikelist = self.dikelist

In [ ]:
dikeNetwork = DikeNetwork()

In [ ]:
dikeNetwork.calc_wave()

In [ ]:
G = dikeNetwork.getG()

In [ ]:
#G.nodes['A.1']

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(G.node['A.0']['Qout'])
df = pd.DataFrame({'Qin':G.node['A.0']['Qout']})

dikelist = dikeNetwork.dikelist
for n in range(0, len(dikelist)):
    node = G.node[dikelist[n]]
    plt.plot(node['Qin'])
    df[dikelist[n]] = node['Qin']

In [ ]:
df